In [1]:
from operator import add

from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.ml.feature import RFormula
from pyspark.mllib.stat import Statistics
from pyspark.sql import Row
from pyspark.sql.types import *

from feat_sel.univariate import SelectKBest

# Functions

# Load test data

In [2]:
# Load data
wwine_rdd = sc.textFile("wine-data/winequality-white.csv")
wwine_rdd = wwine_rdd.map(lambda line: line.replace('"',''))
wwine_rdd = wwine_rdd.map(lambda line: line.replace(';',','))

In [3]:
# Get header
header_str = wwine_rdd.first()
header = header_str.split(',')
header = map(lambda h: h.replace(' ','_'),header)
Wine = Row(*header)


In [4]:
# Create colection of Rows
data = wwine_rdd.filter(lambda x: x!=header_str)
data = data.map(lambda w: w.split(','))
data = data.map(lambda w: map(float,w))
#data = data.map(lambda w: Wine(*w))

# Create DataFrame
df = sqlContext.createDataFrame(data, schema=header)
df.printSchema()

root
 |-- fixed_acidity: double (nullable = true)
 |-- volatile_acidity: double (nullable = true)
 |-- citric_acid: double (nullable = true)
 |-- residual_sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free_sulfur_dioxide: double (nullable = true)
 |-- total_sulfur_dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: double (nullable = true)



In [43]:
# Build feature assemble
predictors = ['fixed_acidity',
              'volatile_acidity',
              'citric_acid',
              'residual_sugar',
              'chlorides',
              'free_sulfur_dioxide',
              'total_sulfur_dioxide',
              'density',
              'pH',
              'sulphates',
              'alcohol']
formula = RFormula(
    formula = 'quality ~ ' + '+'.join(predictors),
    featuresCol = 'features',
    labelCol = 'target'
)
out_df = formula.fit(df).transform(df)

# Univariate selection

In [164]:
# Functions
def d_corr(v1, v2):
    return Statistics.corr(v1,v2)

def d_ftest(v, t):
    
    # calculate auxiliary variables
    n_samples = v.count()
    n_groups = t.distinct().count() # number of distinct groups
    overall_mean = vector.mean() # overall mean
    aux_mean = z_vector.aggregateByKey((0,0),
                                  lambda x,y: (x[0]+y,x[1]+1),
                                  lambda x,y: (x[0]+y[0],x[1]+y[1]))
    group_count = aux_mean.map(lambda (label,x): (label,x[1])) # per group mean
    group_mean = aux_mean.map(lambda (label,x): (label,x[0]/x[1])) # per group mean
    aux_within = z_vector.leftOuterJoin(group_mean)

    # between-group variability
    num = sum([nx[1]*(mx[1]-overall_mean)**2 for (nx,mx) in zip(group_count.collect(),
                                                                group_mean.collect())])/float(n_groups-1)
    
    # within-group variability
    den = aux_within.map(lambda (_,x): (x[0]-x[1])**2).reduce(add)/float(n_samples-n_groups)
    
    return num/den

In [165]:
# Extract features and target
feats = out_df.select('features').rdd
feats = feats.map(lambda x: x['features'])
target = out_df.select('target').rdd
target = target.map(lambda x: x['target'])

# Compute correlation
corr_scores = []
for feat in range(feats.first().size):
    vector = feats.map(lambda x: x[feat])
    corr_scores.append(d_corr(vector,target))
    
f_scores = []
for feat in range(feats.first().size):
    vector = feats.map(lambda x: x[feat])
    f_scores.append(d_ftest(vector,target))

In [5]:
kb = SelectKBest(k=3, method='corr')

feature_cols = ['fixed_acidity',
              'volatile_acidity',
              'citric_acid',
              'residual_sugar']
target_col = 'quality'
kb.fit(df, featureCols=feature_cols, targetCol=target_col)

In [6]:
out_df = kb.transform(df)

In [7]:
out_df.show()

+----------------+-------------+--------------+-----------+
|volatile_acidity|fixed_acidity|residual_sugar|citric_acid|
+----------------+-------------+--------------+-----------+
|            0.27|          7.0|          20.7|       0.36|
|             0.3|          6.3|           1.6|       0.34|
|            0.28|          8.1|           6.9|        0.4|
|            0.23|          7.2|           8.5|       0.32|
|            0.23|          7.2|           8.5|       0.32|
|            0.28|          8.1|           6.9|        0.4|
|            0.32|          6.2|           7.0|       0.16|
|            0.27|          7.0|          20.7|       0.36|
|             0.3|          6.3|           1.6|       0.34|
|            0.22|          8.1|           1.5|       0.43|
|            0.27|          8.1|          1.45|       0.41|
|            0.23|          8.6|           4.2|        0.4|
|            0.18|          7.9|           1.2|       0.37|
|            0.16|          6.6|        